In [36]:
from __future__ import print_function
from traitlets.utils.descriptions import describe
import six
import types
import inspect
import functools
from traitlets import *

#### Decorators

In [37]:
def my_stringify(my_function):
    print('my_function', my_function)
    print('entered my_stringify')
    def my_wrapper(*args, **kwargs):
        print('enter my_wrapper')
        result = my_function(*args, **kwargs)
        print('result computed')
        return str(result)
    print('return my_wrapper')
    return my_wrapper

In [38]:
def my_result(my_function):
    print('my_function', my_function)
    print('entered my_result')
    def my_wrapper(*args, **kwargs):
        print('entered my_wrapper')
        result = my_function(*args, **kwargs)
        print('result computed')
        return result
    print('return my_wrapper')
    return my_wrapper

In [39]:
@my_stringify
def my_square_function(x):
    return x**2

my_function <function my_square_function at 0x10e2ff048>
entered my_stringify
return my_wrapper


In [40]:
my_square_function(2)

enter my_wrapper
result computed


'4'

In [41]:
@my_result
def my_cube_function(x):
    return x**3

my_function <function my_cube_function at 0x10e2ff1e0>
entered my_result
return my_wrapper


In [42]:
my_cube_function(3)

entered my_wrapper
result computed


27

#### Descriptors

In [43]:
class my_descriptor(object):
    
    def __set__(self, obj, value):
        print("setting %r to %r" % (value, obj))

    def __get__(self, obj, cls):
        print("getting from %r" % obj)

    def __delete__(self, obj):
        print("deleting from %r" % obj)

In [44]:
class my_class(object):
    my_x = my_descriptor()

In [45]:
my_object = my_class()

In [46]:
my_object.my_x

getting from <__main__.my_class object at 0x10e3035f8>


In [47]:
my_object.my_x = 5

setting 5 to <__main__.my_class object at 0x10e3035f8>


In [48]:
my_object.my_x = 6

setting 6 to <__main__.my_class object at 0x10e3035f8>


In [49]:
my_object.my_x = 'a value'

setting 'a value' to <__main__.my_class object at 0x10e3035f8>


In [50]:
del my_object.my_x

deleting from <__main__.my_class object at 0x10e3035f8>


#### Metaclasses

In [51]:
for cls in (int, str, bool, object, type):
    print("It is %s that %r is an instance of %r" % (isinstance(cls, type), cls, type))

It is True that <class 'int'> is an instance of <class 'type'>
It is True that <class 'str'> is an instance of <class 'type'>
It is True that <class 'bool'> is an instance of <class 'type'>
It is True that <class 'object'> is an instance of <class 'type'>
It is True that <class 'type'> is an instance of <class 'type'>


In [52]:
class Foobar:
    pass

print('type(Foobar):', type(Foobar))
foo = Foobar()
print('foo: ', foo)
print('type(foo): ', foo)
print('isinstance(foo, Foobar): ', isinstance(foo, Foobar))
#idk why this statement is false
print('isinstance(Foobar, type):', isinstance(Foobar, type))

type(Foobar): <class 'type'>
foo:  <__main__.Foobar object at 0x10e0088d0>
type(foo):  <__main__.Foobar object at 0x10e0088d0>
isinstance(foo, Foobar):  True
isinstance(Foobar, type): True


In [53]:

"""
__dict__: The namespace supporting arbitrary function attributes.

A module object has a namespace implemented by a dictionary object 
(this is the dictionary referenced by the __globals__ attribute of 
functions defined in the module). Attribute references are translated 
to lookups in this dictionary, e.g., m.x is equivalent to m.__dict__["x"].

Attribute assignment updates the module’s namespace 
dictionary, e.g., m.x = 1 is equivalent to m.__dict__["x"] = 1.

Special read-only attribute: __dict__ is the module’s 
namespace as a dictionary object.
"""
class Feature(object):
    
    klass = None
    
    def class_init(self, cls, name):
        print("the feature %r was assigned to %r" % (self, name))
        self.this_class = cls
        self.name = name

    def __set__(self, obj, value):
        if isinstance(value, self.klass):
            obj.__dict__[self.name] = value
        else:
            raise TypeError("Expected a %r instance, not %s" % (self.klass, describe("the", value)))
    
    def __get__(self, obj, cls):
        if obj is None:
            # this is true when accessing the attributed
            # from the class, not instances of the class
            return self
        return obj.__dict__[self.name]
    
    def __delete__(self, obj):
        del obj.__dict__[self.name]



In [54]:
class MetaHasFeatures(type):

    def __init__(cls, name, bases, classdict):
        print("creating %r" % cls)
        for k, v in classdict.items():
            if isinstance(v, Feature):
                v.class_init(cls, k)


In [55]:
class HasFeatures(six.with_metaclass(MetaHasFeatures, object)):
    pass

creating <class '__main__.HasFeatures'>


In [56]:
"""
MyInt inherits Feature
klass is overriden -> only accepts ints
"""
class MyInt(Feature):
    # this feature only accepts ints
    klass = int

In [57]:
"""
MyStr inherits Feature
klass is overriden -> only accepts ints
"""
class MyStr(Feature):
    # this feature only accepts strings
    klass = str

In [58]:
class MyBool(Feature):
    # this feature only accepts boolean
    klass = bool

In [59]:
"""
MyClass inherits HasFeatures
six.with_metaclass(metaclass, *bases)
Create a new class with base classes bases and metaclass metaclass. 
This is designed to be used in class declarations
"""
class MyClass(HasFeatures):
    i = MyInt()
    s = MyStr()
    b = MyBool()
mc = MyClass()

creating <class '__main__.MyClass'>
the feature <__main__.MyInt object at 0x10e303080> was assigned to 'i'
the feature <__main__.MyStr object at 0x10e303fd0> was assigned to 's'
the feature <__main__.MyBool object at 0x10e303128> was assigned to 'b'


In [60]:


# try:
#     mc.i = 1
# except Exception as e:
#     print(e)

# try:
#     mc.s = "i = 1"
# except Exception as e:
#     print(e)
    
# try:
#     mc.b = True
# except Exception as e:
#     print(e)


In [61]:
print("-------------------------------")
mc.i = 1
print("assigned the value %r" % 1)

print("-------------------------------")
mc.s = "i = 1"
print("assigned the value %s" % "i = 1")

print("-------------------------------")
mc.b = True
print("assigned the value %r" % True)

-------------------------------
assigned the value 1
-------------------------------
assigned the value i = 1
-------------------------------
assigned the value True


In [62]:
print("-------------------------------")
print("get %r from %r" % (mc.i, mc))
print("-------------------------------")
print("get %s from %r" % (mc.s, mc))
print("-------------------------------")
print("get %r from %r" % (mc.b, mc))

-------------------------------
get 1 from <__main__.MyClass object at 0x10e303f98>
-------------------------------
get i = 1 from <__main__.MyClass object at 0x10e303f98>
-------------------------------
get True from <__main__.MyClass object at 0x10e303f98>


#### Let's go back to decorators...

A `BaseDecorator` object. When called, it must still act like a method.

In [63]:
class BaseDecorator(object):

    def __init__(self, name, method):
        print('BaseDecorator __init__')
        self.feature_name = name
        self.method = method
    
    def __call__(self, *args, **kwargs):
        print('BaseDecorator __call__')
        return self.method(*args, **kwargs)
    
    def __get__(self, obj, cls):
        print('BaseDecorator __get__')
        if obj is None:
            return self
        return types.MethodType(self.method, obj)

An `observe` decorator. Wraps a method in an `Observer` object.

In [64]:
def observe(name):
    print('observe decorator')
    def setup(method):
        print('observe decorator setup')
        return Observer(name, method)
    return setup

An `Observer` object. After the instance is created it must register itself as an observer.

In [65]:
class Observer(BaseDecorator):
    print('class Observer')
    def instance_init(self, obj):
        print('instance_init')
        obj.observe(self, self.feature_name)
        print('obj.observe')

class Observer


An `OtherFeature` descriptor. It will notify its owner whenever a value changes

In [66]:
class OtherFeature(Feature):
    print('OtherFeature')
    def __set__(self, obj, new):
        print('OtherFeature __set__')
#         print('old: ', old)
        print('obj.__dict__', obj.__dict__)
        old = obj.__dict__.get(self.name, None)
        print('obj.__dict__', obj.__dict__)
        super(OtherFeature, self).__set__(obj, new)
        if new != old:
            data = dict(old=old, new=new, name=self.name, owner=obj)
            obj.notify(data)

OtherFeature


A new `HasOtherFeatures` class. 

It will find `BaseObservers` and call their `instance_init` methods so they can register themselves Offer a method `observe` that will register observers Offer a `notify` which will be used to pass change data to observers.

In [67]:
class HasOtherFeatures(HasFeatures):
    print('HasOtherfeatures')

    def __init__(self):
        print('HasOtherfeatures __init__')
        self._observers = {}
        self.setup()
    
    def setup(self):
        print('HasOtherfeatures setup')
        for k, v in inspect.getmembers(type(self)):
            if isinstance(v, BaseDecorator):
                v.instance_init(self)
    
    def observe(self, handler, name):
        print('HasOtherfeatures observe')
        if name in self._observers:
            olist = self._observers[name]
        else:
            olist = []
            self._observers[name] = olist
        if handler not in olist:
            olist.append(handler)
    
    def notify(self, data):
        print('HasOtherfeatures notify')
        for o in self._observers.get(data["name"], []):
            if isinstance(o, BaseDecorator):
                o(self, data)
            else:
                o(data)

HasOtherfeatures
creating <class '__main__.HasOtherFeatures'>


In [68]:
class MyInt(OtherFeature):
    # this feature only accepts ints
    klass = int
    
class MyStr(OtherFeature):
    # this feature only accepts strings
    klass = str
    
class MyBool(OtherFeature):
    # this feature only accepts boolean
    klass = bool

class MyClass(HasOtherFeatures):
    i = MyInt()
    s = MyStr()
    b = MyBool()
    
    @observe("i")
    def _i_observer(self, data):
        print("observed a change!")
        print(data)

print("-------------------------------")

mc = MyClass()
mc.i = 1
print("-------------------------------")
mc.i = 2
print("-------------------------------")
mc.s = 'i = 2'
print("-------------------------------")
mc.b = True
print("-------------------------------")

observe decorator
observe decorator setup
BaseDecorator __init__
creating <class '__main__.MyClass'>
the feature <__main__.MyInt object at 0x10e3190f0> was assigned to 'i'
the feature <__main__.MyStr object at 0x10e3194e0> was assigned to 's'
the feature <__main__.MyBool object at 0x10e319128> was assigned to 'b'
-------------------------------
HasOtherfeatures __init__
HasOtherfeatures setup
BaseDecorator __get__
instance_init
HasOtherfeatures observe
obj.observe
OtherFeature __set__
obj.__dict__ {'_observers': {'i': [<__main__.Observer object at 0x10e319dd8>]}}
obj.__dict__ {'_observers': {'i': [<__main__.Observer object at 0x10e319dd8>]}}
HasOtherfeatures notify
BaseDecorator __call__
observed a change!
{'old': None, 'new': 1, 'name': 'i', 'owner': <__main__.MyClass object at 0x10e314160>}
-------------------------------
OtherFeature __set__
obj.__dict__ {'_observers': {'i': [<__main__.Observer object at 0x10e319dd8>]}, 'i': 1}
obj.__dict__ {'_observers': {'i': [<__main__.Observer o

#### Traitlets

In [ ]:
class MyClass(HasTraits):

    i = Int()
    j = Int()

    @default("i")
    def _i_default(self):
        """The default value for the trait 'i'
        """
        print("generating default value")
        return 0
    
    @validate("i")
    def _i_validation(self, proposal):
        """A "cross validator" for the trait 'i'
        """
        print("cross validating %r" % proposal.value)
        return proposal.value
    
    @observe("i", type="change")
    def _i_observer(self, change):
        """An observer for the trait 'i'
        """
        print("observed a change from %r to %r" % (change.old, change.new))
        
    
    @default("j")
    def _j_default(self):
        """The default value for the trait 'j'
        """
        print("generating default value")
        return 0
    
    @validate("j")
    def _j_validation(self, proposal):
        """A "cross validator" for the trait 'j'
        """
        print("cross validating %r" % proposal.value)
        return proposal.value
    
    @observe("j", type="change")
    def _j_observer(self, change):
        """An observer for the trait 'j'
        """
        print("observed a change from %r to %r" % (change.old, change.new))

In [35]:
mc = MyClass()

try:
    mc.j = "6"
except TraitError:
    print("mc.j get an error assigning strings")
print("-------------------------------")

try:
    mc.i = "1"
except TraitError:
    print("mc.i get an error assigning strings")

print("i: got the default value %r" % mc.i)
print("-------------------------------")

print("j: got the default value %r" % mc.j)
print("-------------------------------")

print("-------------------------------")
mc.i = 1
# print("assigned, but no change occured")
print("-------------------------------")
mc.j = 2
print("-------------------------------")

HasOtherfeatures __init__
HasOtherfeatures setup
BaseDecorator __get__
instance_init
HasOtherfeatures observe
obj.observe
-------------------------------
OtherFeature __set__
obj.__dict__ {'_observers': {'i': [<__main__.Observer object at 0x10dff00f0>]}, 'j': '6'}
obj.__dict__ {'_observers': {'i': [<__main__.Observer object at 0x10dff00f0>]}, 'j': '6'}


NameError: name 'TraitError' is not defined

# Traitlets
---

## Defaults

Default value generators create values "on request". In other words, it's only when you call `getattr`
that the method `MyClass._i_default` will be used to create the default. Whatever it returns will be validated, assigned as the current value of the trait, and then returned to the getattr call.

## Validation

Traits have internal validation and cross validation. The interal stages occur first, and are the same for all instances of a trait type. Cross validators are created by users with the `@validate` decorator - we call them this because a validator which is a method on the owner of the trait can check to see whether the value is correct with respect to the state of the instance itself - say to check whether a value falls between the `min` and `max` attributes on the owner instance. Cross validation occurs after internal validation, so a cross validator of an `Int` trait can already expect that the values it recieves are `int` instances. No need to check whether `isinstance(proposal.value, int)`.

## Observers

Events can be observed by specifying trait names, traits with certain metadata, and/or events of a certain type. These events are triggered by passing a bunch to the method `HasTraits.notify_change`. So if you need to create your own kinds of events that people can listen into you need only define a bunch with a 'name' and a 'type'.

In [ ]:
def on_the_fly_observer(change):
    print("caught the same change on the fly" % change)

mc.observe(on_the_fly_observer, "i")
mc.i = 3